In [12]:
import configparser
from pathlib import Path
from datetime import datetime
from os import PathLike

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [13]:
def read_config(fn):
    # reads in an ini config file, including any blank lines

    # read in file, add "Null" to any empty inputs
    fn = Path(fn)
    with open(fn, 'r') as f:
        lines = f.readlines()
    for i, _ in enumerate(lines):
        if lines[i][-2:] == '=\n':
            lines[i] = lines[i][:-1] + 'Null' + lines[i][-1]
    complete_fn = fn.parent / (fn.stem + '_complete.eddypro')
    with open(complete_fn, 'w') as f:
        for ln in lines:
            f.write(ln)

    ini = configparser.ConfigParser()
    ini.read(complete_fn)
    return ini

def read_config_as_dataframe(fn):
    lines = []
    ini = read_config(fn)
    for section in ini.sections():
        for option, value, in ini[section].items():
            lines.append([section, option, value])
    df = pd.DataFrame(lines, columns=['Section', 'Option', 'Value'])
    df = df.sort_values(['Section', 'Option'])
    df['Name'] = fn.stem
    return df

def compare_configs(df1: pd.DataFrame, df2: pd.DataFrame):
    # compare differences between two config dataframe
    df1_new = df1.loc[df1['Value'] != df2['Value'], ['Section', 'Option', 'Value']]
    df2_new = df2.loc[df1['Value'] != df2['Value'], ['Section', 'Option', 'Value']]
    name1 = df1['Name'].values[0]
    name2 = df2['Name'].values[0]
    df_compare = (
        df1_new
        .merge(df2_new, on=['Section', 'Option'], suffixes=['_'+name1, '_'+name2])
        .sort_values(['Section', 'Option'])
    )
    return df_compare

In [14]:
environment = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro')

# When we hit buttons in EddyPro, what settings change in the .ini file?


## Set up a template
First, we create a reference .eddypro file. This file will have all the default processing settings.

Note that EddyPro will leave some options blank

In [15]:
base = read_config_as_dataframe(environment / 'ini/base.eddypro')
base.head()

,Section,Option,Value,Name
0,FluxCorrection_SpectralAnalysis_General,add_sonic_lptf,1,base
1,FluxCorrection_SpectralAnalysis_General,ex_dir,Null,base
2,FluxCorrection_SpectralAnalysis_General,ex_file,Null,base
3,FluxCorrection_SpectralAnalysis_General,horst_lens,2,base
4,FluxCorrection_SpectralAnalysis_General,sa_bin_spectra,Null,base


## Basic Settings
### Changing dates
change **Basic-Settings/Start** and **Basic-Settings/End**

In [16]:
change_dates = read_config_as_dataframe(environment / 'ini/change_dates.eddypro')
compare_configs(base, change_dates)

,Section,Option,Value_base,Value_change_dates
0,Project,col_co2,0,8
1,Project,col_h2o,0,9
2,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/change_dates.eddypro
3,Project,last_change_date,2023-08-09T16:37:17,2023-07-26T11:11:27
4,Project,pr_end_date,2020-07-22,2020-06-26
5,Project,pr_end_time,00:00,04:30
6,Project,pr_start_date,2020-06-21,2020-06-23
7,Project,pr_start_time,00:00,02:30
8,Project,project_id,base,change_dates
9,RawProcess_Settings,out_bin_sp,1,0


other than trivial changes and housekeeping changes like file_name, last_change_date, and project_id, the only changed settings are the start and end times and dates, as expected:

These are pretty self-explanatory:
* Project/pr_end_time
* Project/pr_start_time
* Project/pr_end_date
* Project/pr_start_date

### Missing Samples Allowance and Flux Averaging Interval
Change **Basic-Settings/Missing-Samples-Allowance** and **Basic-Settings/Flux-Averaging-Interval**

In [17]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/missing-avgint.eddypro')
)


,Section,Option,Value_base,Value_missing-avgint
0,Project,col_co2,0,8
1,Project,col_h2o,0,9
2,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/missing-avgint.eddypro
3,Project,last_change_date,2023-08-09T16:37:17,2023-07-26T13:36:21
4,Project,project_id,base,missing-avgint
5,RawProcess_Settings,avrg_len,30,0
6,RawProcess_Settings,max_lack,10,0
7,RawProcess_Settings,out_bin_sp,1,0
8,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30


Changes to INI file:
* RawProcess_Settings/avrg_len: averaging interval 
* RawProcess_Settings/max_lack: maximum % missing values

### North Reference
Change **Basic-Settings/North-Reference** to Geographic North, but don't change 

In [18]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/northref.eddypro')
)

,Section,Option,Value_base,Value_northref
0,Project,col_co2,0,8
1,Project,col_h2o,0,9
2,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/northref.eddypro
3,Project,last_change_date,2023-08-09T16:37:17,2023-07-26T11:37:33
4,RawProcess_General,dec_date,2023-12-31,2020-07-22
5,RawProcess_General,mag_dec,0,8.666667
6,RawProcess_General,use_geo_north,0,1
7,RawProcess_Settings,out_bin_sp,1,0


* RawProcess_General/dec_date (yyyy-mm-dd)
* RawProcess_General/mag_dec (decimal declination)
* RawProcess_General/use_geo_north (0 -> 1)
* TiltCorrection_Settings/pf_end_time (23:30 -> 00:00??, ignoring for now)

### Wind speed measurement offsets
Change **Advanced-Settings/Wind-Speed-Measurement-Offsets**

In [19]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/windspeedoffsets.eddypro')
)

,Section,Option,Value_base,Value_windspeedoffsets
0,Project,col_co2,0,8
1,Project,col_h2o,0,9
2,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/windspeedoffsets.eddypro
3,Project,last_change_date,2023-08-09T16:37:17,2023-07-26T14:07:54
4,RawProcess_Settings,out_bin_sp,1,0
5,RawProcess_Settings,u_offset,0,1.0999999999999999
6,RawProcess_Settings,v_offset,0,-1.0999999999999999
7,RawProcess_Settings,w_offset,0,-0.30000000000000004
8,RawProcess_TiltCorrection_Settings,pf_end_time,00:00,23:30


### Axis rotations for tilt correction

Unticking tilt corrections: rot_meth=0

Selecting triple rotations: rot_meth=2

Selecting double rotations: rot_meth=1

Selecting planar fit with a provided file: rot_meth=3, RawProcess_TiltCorrection_Settings/pf_mode = 0, pf_file=filepath

Selecting planar fit (no v bias) with a provided file: rot_meth=4, RawProcess_TiltCorrection_Settings/pf_mode = 0, pf_file=filepath

When providing planar fit settings manually, change the following in RawProcess_TiltCorrection_Settings

* pf_start/end_date/time
* pf_subset: 1???
* pf_mode: 1 for manual, 0 for file
* pf_u_min: minimum mean horizontal windspeed [0, 10]
* pf_w_max: maximum mean vertical windspeed (0, 10]
* pf_min_num_per_sec: minimum elements per sector [0, 9999]
* pf_fix: what to do if calculations fail for a sector (closest clockwise 0, closest ccw 1, dr 2)
* pf_north_offset: north offset for minimum azimuth of first sector
* sectors: n goes from 1 to max of 10
  * pf_sector_n_exclude: 1 to ignore for PF, 0 to use for PF
  * pf_sector_n_width: degrees, from 1 to 359

### Extracting Turbulent Fluctuations
This one is in **Advanced-Settings/Processing/Turbulent-fluctuations** and is pretty simple:

RawProcess_Settings/detrend_meth: 0 (block avg), 1 (linear detrending), 2 (running mean), 3 (exponential running mean)

RawProcess_Settings/timeconst: 0 gives same as flux averaging interval, otherwise units are seconds.

### Time lags compensation
Manually configuring auto-optimization

In [20]:
read_config_as_dataframe(environment / 'ini/tlags_complete.eddypro').query('Section == "RawProcess_TimelagOptimization_Settings"').sort_values('Option')

,Section,Option,Value,Name
350,RawProcess_TimelagOptimization_Settings,to_ch4_max_lag,-999.3,tlags_complete
351,RawProcess_TimelagOptimization_Settings,to_ch4_min_flux,0.202,tlags_complete
352,RawProcess_TimelagOptimization_Settings,to_ch4_min_lag,-999.4,tlags_complete
353,RawProcess_TimelagOptimization_Settings,to_co2_max_lag,1000.0,tlags_complete
354,RawProcess_TimelagOptimization_Settings,to_co2_min_flux,1.998,tlags_complete
355,RawProcess_TimelagOptimization_Settings,to_co2_min_lag,-998.4,tlags_complete
356,RawProcess_TimelagOptimization_Settings,to_end_date,2020-07-22,tlags_complete
357,RawProcess_TimelagOptimization_Settings,to_end_time,02:00,tlags_complete
358,RawProcess_TimelagOptimization_Settings,to_file,/Users/alex/Documents/Work/UWyo/2023 Summer/Fluxcourse/Project/eddypro_output/lostcreek/Cov Max Auto Opt/eddypro_TimeLag-AutomaticOpt_timelag_opt_2023-06-28T220023_adv.txt,tlags_complete
359,RawProcess_TimelagOptimization_Settings,to_gas4_max_lag,-999.0,tlags_complete


RawProcess_Settings/tlag_meth:
* No compensation: tlag_meth = 0
* Constant: tlag_meth = 1, as specified in metadata
* Covariance maximization with default: 2
* Naive maximization: 3
* Automatic time lag optimization: provide a file or manually modify settings: 4
    * provide a file:
      * RawProcess_TimeLagOptimization_Settings to_file = path to timelag_opt output file
      * RawProcess_TimeLagOptimization_Settings to_mode = 0
      * RawProcess_TimeLagOptimization_Settings to_subset = 0
    * manually input settings:
      * RawProcess_TimeLagOptimization_Settings to_subset = 0 or 1?????
      * RawProcess_TimeLagOptimization_Settings to_mode = 1
      * to_co2/ch4/h2o/gas4_max/min_lag: -1000s to +1000s, max > min
      * to_start/end_date/time
      * etc...(FINISH)



### Compensate Density Fluctuations

Disable/enable WPL and burba corrections, and fiddle with burba settings

In [21]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/densfluct.eddypro')
)

,Section,Option,Value_base,Value_densfluct
0,Project,col_co2,0,8
1,Project,col_h2o,0,9
2,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/densfluct.eddypro
3,Project,files_found,1386,0
4,Project,last_change_date,2023-08-09T16:37:17,2023-08-08T18:05:14
5,RawProcess_Settings,bu_corr,0,1
6,RawProcess_Settings,bu_multi,0,1
7,RawProcess_Settings,m_day_bot1,2.80000000,2.81000000
8,RawProcess_Settings,m_day_bot2,-0.06810000,1.00000000
9,RawProcess_Settings,m_day_bot3,0.00210000,1.00000000


Disabling Compensate density fluctuations:
* Project/wpl_meth 1->0

Enable instr. sensible head components
* RawProcess_Settings/bu_corr 0->1
* Simple linear vs multiple reg: Rawprocess_settings/bu_multi 0->1
* Changing offsets (RawProcess_Settings)
  * l_day/night_bot/top/spar_gain/offset for simple linear
  * m_day/night_bot/top/spar_1/2/3/4 for MLR



### Statistical Analysis

In [22]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/statanal.eddypro')
)

,Section,Option,Value_base,Value_statanal
0,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/statanal.eddypro
1,Project,files_found,1386,0
2,Project,last_change_date,2023-08-09T16:37:17,2023-08-11T15:05:06
3,RawProcess_ParameterSettings,despike_vm,0,1
4,RawProcess_Settings,detrend_meth,0,1
5,RawProcess_Settings,rot_meth,1,4
6,RawProcess_Settings,timeconst,250.0,0.0
7,RawProcess_Settings,tlag_meth,2,4
8,RawProcess_TimelagOptimization_Settings,to_ch4_min_flux,0.200,100.000
9,RawProcess_TimelagOptimization_Settings,to_co2_max_lag,-1000.1,999.4


enabling/disabling statistical tests:

* despike: RawProcess_ParameterSettings/despike_vm 
* despiking: RawProcess_Tests/test_sr 0/1
  * when using vickers + mahrt 97: RawProcess_ParameterSettings/despike_vm = 0. 1 for mauder 2013. Confusing I know
  * spikes: RawProcess_ParameterSettings/sr_lim_hf
  * linterp:  RawProcess_Settings/filter_sr 1 for linterp, 0 for no linterp
  * limits: a little complicated. basic format is sr_lim_name:
    * w: sr_lim_w
    * co2: sr_lim_co2
    * h2o: sr_lim_h2o
    * ch4: sr_lim_ch4
    * gas4: sr_lim_n2o
    * all other variables: sr_lim_u
    * there's also another secret setting, sr_lim_hf that seems to be always set to 2. Perhaps leave this alone?
  * outliers consecutive: sr_num_spk
  * limits:
    * accepted spikes: 0-50%
    * consec outliers: 3-1000
    * z-scores: 1-20
* Abs lims RawProcess_Tests/test_al
  * pretty straightforward: al_vanam_min/max for tson, u, w, h2o, co2, ch4, n2o (for gas4). u and v only have max
  * RawProcess_ParameterSettings/filter_al 1 for filter outranged vals, 0 for don't filter
  * max/min:
    * U: 1 to 50
    * W: 0.5 to 10
    * T: -100 to 100
    * CO2: 100 to 10_000
    * H2O: 0 to 1000
    * CH4: 0 to 1000
    * gas4: 0 to 1000
* Amplitude resolution: RawProcess_Tests/test_ar 0/1
  * RawProcess_ParameterSettings/ar_bins for num bins, ar_hf_lim for % empty bins, ar_lim for range of variation
  * Limits:
    * Range of variation: 1-20
    * bins: 50-150
    * empty %: 1-100
* Dropouts: RawProcess_Tests/test_do 0/1
  * RawProcess_ParameterSettings/do_extlim_dw for extreme percentile
  * RawProcess_ParameterSettings/do_hf1_lim for % central, do_hf2_lim for %extreme
* Skewness/kurtosis: RawProcess_Tests/test_sk 0/1
  * straightforward: RawProcess_ParameterSettings/sk_hf/sf_sk/kumin/max. hf for hard, sf for soft, sk for skew, ku for kurt.
  * note that softflag has to be <= in magnitude than hard flag. Skewness lower must be in [-3, -0.1]. Skewness upper in [0.1, 3]. Kurt lower in [0.1, 3] and upper in [3, 10]
* Discontinnuities: RawProcess_Tests/test_ds
  * RawProcess_ParameterSettings/ds_hf/sf_varname, where varname is
    * sonic t: t
    * u: uv
    * w: w
    * variance: var
    * co2/h2o/ch4/n2o
  * limits: 0-50 for everything, with hardflag >= softflag
* Timelags: RawProcess_Tests/test_tl
  * nominal timelags: RawProcess_ParameterSettings/tl_def_ch4/co2/h2o/n2o
  * flags: RawProcess_ParameterSettings/tl_hf/sf_lim
  * note: can get nominal time lags from metadata?
  * limits: 
    * hard > soft, 0-100%
    * 0-100s
* AoA: RawProcess_Tests/aa
  * aa_lim, aa_max, aa_min
  * aa_max, aa_min 0-90 each (-90-0 for min)
  * 0-100%
* Steadiness: RawProcess_ParameterSettings/test_ns
  * RawProcess_ParameterSettings/ns_hf_lim 0-50
* Random uncertainty: 
  * Project/ru_meth
    * 0: none
    * 1: Finkle
    * 2: Mann
    * 3: Mahrt
  * Its def: Project/ru_its_meth
    * 0: 1/e
    * 1: 0-cros
    * 2: corr period
  * Project/ru_tlag_max for window width, 0-10_000s

## Spectral corrections

In [35]:
compare_configs(
    base, 
    read_config_as_dataframe(environment / 'ini/spectra.eddypro')
)

,Section,Option,Value_base,Value_spectra
0,FluxCorrection_SpectralAnalysis_General,sa_hfn_co2_fmin,1.0000,50.0000
1,FluxCorrection_SpectralAnalysis_General,sa_hfn_gas4_fmin,1.0000,1.1000
2,Project,bin_sp_avail,0,1
3,Project,file_name,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/base.eddypro,/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/investigate_eddypro/ini/spectra.eddypro
4,Project,last_change_date,2023-08-09T16:37:17,2023-08-16T18:57:56
